# Fligths analysis - preprocessing continuation - set "a" and "b"

## Import libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

## Load preprocessed data

In [2]:
data_preprocessed_a = pd.read_csv("airlines_delay_preprocessed_a.csv")
data_preprocessed_a.head()

,Time,Length,AirlineReg,AirlineCheap,AirportFrom,AirportTo,DayOfWeek,Class
0,1296.0,141.0,0,0,0,4,1,0
1,360.0,146.0,1,0,4,1,4,0
2,1170.0,143.0,0,1,3,2,3,0
3,1410.0,344.0,0,0,4,2,6,0
4,692.0,98.0,0,1,6,0,4,0


In [3]:
data_preprocessed_b = pd.read_csv("airlines_delay_preprocessed_b.csv")
data_preprocessed_b.head()

,Time,Length,Airline,AirportFrom,AirportTo,DayOfWeek,Class
0,1296.0,141.0,1,0,4,1,0
1,360.0,146.0,2,4,1,4,0
2,1170.0,143.0,6,3,2,3,0
3,1410.0,344.0,4,4,2,6,0
4,692.0,98.0,5,6,0,4,0


## Select inputs and targets

In [4]:
targets_a = data_preprocessed_a["Class"]
inputs_a = data_preprocessed_a.iloc[:,:-1]

In [5]:
targets_b = data_preprocessed_b["Class"]
inputs_b = data_preprocessed_b.iloc[:,:-1]

## Select dummies

In [6]:
def select_dummies(data):
    col_dummies = []
    for column in data.columns:
        vc = data[column].value_counts()
        comp = vc.values.tolist()
        s = data[column].sum()
        # Compare the number of 1s in the column to the sum of all values
        if comp[1] == s:
            col_dummies.append(column)
        
    return col_dummies

In [7]:
columns_with_dummies_a = select_dummies(inputs_a)
columns_with_dummies_a

['AirlineReg', 'AirlineCheap']

In [8]:
columns_with_dummies_b = select_dummies(inputs_b)
columns_with_dummies_b

[]

In [9]:
columns_without_dummies_a = inputs_a.drop(columns_with_dummies_a, axis=1)
columns_without_dummies_a

,Time,Length,AirportFrom,AirportTo,DayOfWeek
0,1296.0,141.0,0,4,1
1,360.0,146.0,4,1,4
2,1170.0,143.0,3,2,3
3,1410.0,344.0,4,2,6
4,692.0,98.0,6,0,4
...,...,...,...,...,...
539377,530.0,72.0,4,3,5
539378,560.0,115.0,2,2,4
539379,827.0,74.0,5,0,2
539380,715.0,65.0,3,4,4


In [10]:
columns_without_dummies_b = inputs_b.drop(columns_with_dummies_b, axis=1)
columns_without_dummies_b

,Time,Length,Airline,AirportFrom,AirportTo,DayOfWeek
0,1296.0,141.0,1,0,4,1
1,360.0,146.0,2,4,1,4
2,1170.0,143.0,6,3,2,3
3,1410.0,344.0,4,4,2,6
4,692.0,98.0,5,6,0,4
...,...,...,...,...,...,...
539377,530.0,72.0,2,4,3,5
539378,560.0,115.0,0,2,2,4
539379,827.0,74.0,5,5,0,2
539380,715.0,65.0,0,3,4,4


## Select outliers

In [11]:
# Define function to find outliers and calculate the percentage of them in each column
def evaluate_outliers(data):
    outliers_percents = {}
    for column in data.columns:
        if data[column].dtype!=object:
            q1 = np.quantile(data[column], 0.25)
            q3 = np.quantile(data[column], 0.75)
            iqr = q3 - q1
            upper_bound = q3 + (1.5*iqr)
            lower_bound = q1 - (1.5*iqr)
            outliers = data[(data[column]>upper_bound) | (data[column]<lower_bound)][column]
            outliers_percentage = len(outliers)/len(data[column])*100
            outliers_percents[column] = outliers_percentage
            outliers_dataframe = pd.DataFrame(data=outliers_percents.values(), 
                                             index=outliers_percents.keys(), 
                                             columns=['Outlier percentage']
                                            )
    
    return outliers_dataframe.sort_values(by='Outlier percentage', ascending=False)

In [12]:
df_no_dummies_a = evaluate_outliers(columns_without_dummies_a)
df_no_dummies_a

,Outlier percentage
Length,4.755442
Time,0.000000
AirportFrom,0.000000
AirportTo,0.000000
DayOfWeek,0.000000


In [13]:
df_no_dummies_b = evaluate_outliers(columns_without_dummies_b)
df_no_dummies_b

,Outlier percentage
Length,4.755442
Time,0.000000
Airline,0.000000
AirportFrom,0.000000
AirportTo,0.000000
DayOfWeek,0.000000


In [14]:
#Define function which select columns with outliers
def select_outliers(data):
    outliers = []
    for column in data.columns:
        for i in range(len(data)):
            k = data[column].index.tolist()
            if data[column][i]>0.0:
                outliers.append(k[i])
                
    return outliers

In [15]:
columns_with_outliers_a = select_outliers(df_no_dummies_a)
columns_with_outliers_a

['Length']

In [16]:
columns_with_outliers_b = select_outliers(df_no_dummies_b)
columns_with_outliers_b

['Length']

In [17]:
inputs_a.columns.values

array(['Time', 'Length', 'AirlineReg', 'AirlineCheap', 'AirportFrom',
       'AirportTo', 'DayOfWeek'], dtype=object)

In [18]:
columns_without_outliers_a = [x for x in inputs_a.columns.values if 
                              (x not in columns_with_dummies_a and 
                               x not in columns_with_outliers_a)]

columns_without_outliers_a

['Time', 'AirportFrom', 'AirportTo', 'DayOfWeek']

In [19]:
inputs_b.columns.values

array(['Time', 'Length', 'Airline', 'AirportFrom', 'AirportTo',
       'DayOfWeek'], dtype=object)

In [20]:
columns_without_outliers_b = [x for x in inputs_b.columns.values if 
                              (x not in columns_with_dummies_b and
                               x not in columns_with_outliers_b)]

columns_without_outliers_b

['Time', 'Airline', 'AirportFrom', 'AirportTo', 'DayOfWeek']

## Standardize the data

We see we have three types of data:
- dummies
- data without outliers
- data with outliers

Dummies do not have to be standardized, but the other two groups have to be standardized. We will create two types of scalers to standardize the two groups. These scalers are of the general use.

In [21]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, RobustScaler

In [22]:
# Create a class of a standard scaler (for data without outliers):
class CustomScalerStandard(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.scaler = StandardScaler()
        self.columns = columns
        
    def fit(self, x, y=None):
        self.scaler.fit(x[self.columns], y)
        return self
    
    def transform(self, x, y=None):
        init_col_order = x.columns
        x_scaled = pd.DataFrame(self.scaler.transform(x[self.columns]), 
                                columns=self.columns
                               )
        x_not_scaled = x.loc[:,~x.columns.isin(self.columns)]
        return pd.concat([x_not_scaled, x_scaled], axis=1)[init_col_order]

In [23]:
# Create a class of a robust scaler (for data with outliers):
class CustomScalerRobust(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.scaler = RobustScaler()
        self.columns = columns
        
    def fit(self, x, y=None):
        self.scaler.fit(x[self.columns], y)
        return self
    
    def transform(self, x, y=None):
        init_col_order = x.columns
        x_scaled = pd.DataFrame(self.scaler.transform(x[self.columns]), 
                                columns=self.columns
                               )
        x_not_scaled = x.loc[:,~x.columns.isin(self.columns)]
        return pd.concat([x_not_scaled, x_scaled], axis=1)[init_col_order]

In [24]:
# Define and apply scalers for the dataset a:
try:
    flights_scaler_standard = CustomScalerStandard(columns_without_outliers_a)
    flights_scaler_standard.fit(inputs_a)
    inputs_a = flights_scaler_standard.transform(inputs_a)  
except:
    print("No columns to use for data 'a' without outliers.")

try:
    flights_scaler_robust = CustomScalerRobust(columns_with_outliers_a)
    flights_scaler_robust.fit(inputs_a)
    inputs_a = flights_scaler_robust.transform(inputs_a)
except:
    print("No columns to use for data 'a' with outliers.")

In [25]:
inputs_a

,Time,Length,AirlineReg,AirlineCheap,AirportFrom,AirportTo,DayOfWeek
0,1.774070,0.320988,0,0,-1.967403,0.764603,-1.530120
1,-1.592288,0.382716,1,0,0.763943,-1.284407,0.036734
2,1.320907,0.345679,0,1,0.081106,-0.601403,-0.485550
3,2.184075,2.827160,0,0,0.763943,-0.601403,1.081304
4,-0.398238,-0.209877,0,1,2.129615,-1.967410,0.036734
...,...,...,...,...,...,...,...
539377,-0.980877,-0.530864,1,0,0.763943,0.081600,0.559019
539378,-0.872981,0.000000,0,1,-0.601730,-0.601403,0.036734
539379,0.087295,-0.506173,0,1,1.446779,-1.967410,-1.007835
539380,-0.315518,-0.617284,0,1,0.081106,0.764603,0.036734


In [26]:
# Define and apply scalers for the dataset b:
try:
    flights_scaler_standard = CustomScalerStandard(columns_without_outliers_b)
    flights_scaler_standard.fit(inputs_b)
    inputs_b = flights_scaler_standard.transform(inputs_b)  
except:
    print("No columns to use for data 'b' without outliers.")

try:
    flights_scaler_robust = CustomScalerRobust(columns_with_outliers_b)
    flights_scaler_robust.fit(inputs_b)
    inputs_b = flights_scaler_robust.transform(inputs_b)
except:
    print("No columns to use for data 'b' with outliers.")

In [27]:
inputs_b

,Time,Length,Airline,AirportFrom,AirportTo,DayOfWeek
0,1.774070,0.320988,-1.037651,-1.967403,0.764603,-1.530120
1,-1.592288,0.382716,-0.562929,0.763943,-1.284407,0.036734
2,1.320907,0.345679,1.335959,0.081106,-0.601403,-0.485550
3,2.184075,2.827160,0.386515,0.763943,-0.601403,1.081304
4,-0.398238,-0.209877,0.861237,2.129615,-1.967410,0.036734
...,...,...,...,...,...,...
539377,-0.980877,-0.530864,-0.562929,0.763943,0.081600,0.559019
539378,-0.872981,0.000000,-1.512373,-0.601730,-0.601403,0.036734
539379,0.087295,-0.506173,0.861237,1.446779,-1.967410,-1.007835
539380,-0.315518,-0.617284,-1.512373,0.081106,0.764603,0.036734


## Saving inputs and targets

In [30]:
inputs_a.to_csv("inputs_a.csv", index=False)
targets_a.to_csv("targets_a.csv", index=False)

In [29]:
inputs_b.to_csv("inputs_b.csv", index=False)
targets_b.to_csv("targets_b.csv", index=False)